Find the residuals between ICESat-2 elevation measurements and a reference elevation (taken from IceBridge ATM flyover of Zachariae Isstrom)

Taryn Black, ICESat-2 Hackweek, June 17-21 2019

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

# Set data location info

In [2]:
home_dir = "/home/jovyan/xtrak/"
cross_file = "data_prod/InterX_ATM2014_AllSmooth.csv"
ATM_year = '2014'

# Load ICESat-2 crossover points

ICESat-2 track points that were identified as crossing our OIB ATM reference track in Intersections.ipynb. This file contains both the ICESat-2 elevation data and the OIB ATM data at the crossover points.

In [3]:
cross_df = pd.read_csv(home_dir + cross_file, parse_dates=[4])

cross_df.head()
#cross_df.info()

,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06,slope_ATM,slope_ATL06
0,49165.513987,44.89925,1442.0,42.769034,2018-10-18 15:53:52,617767.0,gt1l,0.002337,0.009302
1,49099.625709,44.74530,1440.0,42.156155,2018-10-18 15:53:52,621074.0,gt1r,NaN,NaN
2,45936.554970,36.59180,1343.0,34.448452,2018-10-18 15:53:52,624012.0,gt2l,0.009535,0.024877
3,45871.380221,35.97035,1341.0,32.827094,2018-10-18 15:53:52,626687.0,gt2r,NaN,NaN
4,91172.740382,294.13865,2671.0,NaN,2018-10-21 05:21:45,205271.0,gt2r,NaN,NaN


# Visual check of data

Plot reference elevation profile and crossover data points.

In [4]:
%matplotlib widget

plt.figure(figsize=(12,8))
ax1 = plt.scatter(cross_df['dist_along']/1000, cross_df['ATM_elev'], c='black')
ax2 = plt.scatter(cross_df['dist_along']/1000, cross_df['z_ATL06'], c=cross_df['t_ATL06'], s=12)
plt.xlabel('Distance along track (km)')
plt.ylabel('Elevation (m)')
plt.title('Elevation profiles from ATM and ICESat-2')
plt.colorbar(label='Time')
plt.legend(['%s ATM reference track' % ATM_year]);

FigureCanvasNbAgg()

# Calculate residual between ICESat-2 track crossovers and reference elevation profile

For each point in the ICESat-2 crossover dataframe, subtract the reference elevation at the same distance along-track.

$residual = z_{ICESat2} - z_{reference}$

Thus, positive values indicate an increase in elevation compared to the reference track, and negative values indicate a decrease in elevation.

In [5]:
cross_df['residuals'] = cross_df['z_ATL06'] - cross_df['ATM_elev']

cross_df.head()

,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06,slope_ATM,slope_ATL06,residuals
0,49165.513987,44.89925,1442.0,42.769034,2018-10-18 15:53:52,617767.0,gt1l,0.002337,0.009302,-2.130216
1,49099.625709,44.74530,1440.0,42.156155,2018-10-18 15:53:52,621074.0,gt1r,NaN,NaN,-2.589145
2,45936.554970,36.59180,1343.0,34.448452,2018-10-18 15:53:52,624012.0,gt2l,0.009535,0.024877,-2.143348
3,45871.380221,35.97035,1341.0,32.827094,2018-10-18 15:53:52,626687.0,gt2r,NaN,NaN,-3.143256
4,91172.740382,294.13865,2671.0,NaN,2018-10-21 05:21:45,205271.0,gt2r,NaN,NaN,NaN


Let's plot the residuals!

In [6]:
plt.figure(figsize=(12,4))
plt.scatter(cross_df['dist_along']/1000, cross_df['residuals'], c=cross_df['t_ATL06'])
plt.axhline(0, color='k', lw=0.5)
plt.xlabel('Distance along track (km)')
plt.ylabel('Elevation (m)')
plt.title('Elevation residuals \n (+)=raised, (-)=lowered')
plt.colorbar(label='Time');

FigureCanvasNbAgg()

# Export dataframe to CSV

Exported CSV file includes distance along track, ATM data (elevation, index), ATL06 data (elevation, index, time, groundtrack), and the residual (difference between ATL06 and ATM).

In [7]:
cross_df.to_csv(home_dir + "data_prod/residuals.csv")

# Residual statistics

Let's further explore what's going on in our elevation data.

## Variation in elevation changes, along-track

Bin the residuals by distance along track, and look at the standard deviation of residual values in each bin, using a box plot. Why do this? We hypothesize that we will see more elevation variation near the terminus than farther up-flow.

In [8]:
bin_min = np.around(np.floor(cross_df['dist_along'].min()/1000), -1)
bin_max = np.around(np.ceil(cross_df['dist_along'].max()/1000), -1)
bins = np.arange(bin_min,bin_max,10)
print(bins)

cross_df['dist_binned'] = pd.cut(cross_df['dist_along']/1000, bins)
cross_df.head()

[ 40.  50.  60.  70.  80.  90. 100. 110.]


,dist_along,ATM_elev,idx_ATM,z_ATL06,t_ATL06,idx_ATL06,gt_ATL06,slope_ATM,slope_ATL06,residuals,dist_binned
0,49165.513987,44.89925,1442.0,42.769034,2018-10-18 15:53:52,617767.0,gt1l,0.002337,0.009302,-2.130216,"(40.0, 50.0]"
1,49099.625709,44.74530,1440.0,42.156155,2018-10-18 15:53:52,621074.0,gt1r,NaN,NaN,-2.589145,"(40.0, 50.0]"
2,45936.554970,36.59180,1343.0,34.448452,2018-10-18 15:53:52,624012.0,gt2l,0.009535,0.024877,-2.143348,"(40.0, 50.0]"
3,45871.380221,35.97035,1341.0,32.827094,2018-10-18 15:53:52,626687.0,gt2r,NaN,NaN,-3.143256,"(40.0, 50.0]"
4,91172.740382,294.13865,2671.0,NaN,2018-10-21 05:21:45,205271.0,gt2r,NaN,NaN,NaN,"(90.0, 100.0]"


In [9]:
cross_df.describe()

,dist_along,ATM_elev,idx_ATM,z_ATL06,idx_ATL06,slope_ATM,slope_ATL06,residuals
count,189.000000,189.000000,189.000000,183.000000,189.000000,86.000000,86.000000,183.000000
mean,76043.464870,148.573494,2221.941799,134.242884,364099.042328,0.006545,-0.006391,-9.524648
std,19829.278968,147.459563,583.404235,137.598210,205308.191243,0.016920,0.107088,16.499335
min,43829.598009,27.027150,1279.000000,27.364034,1576.000000,-0.034906,-0.930625,-62.500363
25%,59108.312790,27.363400,1721.000000,30.591542,190415.000000,-0.000000,-0.009278,-23.611458
50%,76094.688011,94.213900,2222.000000,70.762263,373515.000000,0.000000,0.001333,-5.131670
75%,91172.740382,264.995600,2671.000000,240.270607,540515.000000,0.006375,0.015179,1.104613
max,114859.325846,572.425900,3374.000000,540.213215,716129.000000,0.085970,0.127760,41.533753


In [ ]:
crossbox = cross_df.boxplot(by='dist_binned', column='residuals')